# Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
import time
from sklearn.metrics import mean_squared_error, r2_score #avaliar predicao da regressao linear

# Dataset

In [2]:
np.random.seed(42)

x1 = np.random.randint(5, 100, size=50)
x2 = np.random.randint(1, 6, size=50)
x3 = np.random.randint(1, 6, size=50)

y1 = (x1 * 1000) + (x2 * 500) + (x3 * 500) + np.random.randint(0, 500, size=50)

df = pd.DataFrame({
    'metros_quadrados': x1,
    'quartos': x2,
    'banheiros': x3,
    'preco': y1
})

df.head()

,metros_quadrados,quartos,banheiros,preco
0,56,4,3,59561
1,97,1,1,98440
2,19,4,3,22633
3,76,2,3,78783
4,65,2,1,66527


O dataset sintetico representa o preço de imoveis com base no numero de metros quadrados, quartos e banheiros. Foram criados 50 registros. O dataset não possui valores nulos.

# Regressão linear multipla sem Scikit-learn

A regressão linear múltipla modela a relação entre uma variável dependente y e várias variáveis independentes x₁, x₂, ..., xₚ. temos como equação:

### $ŷ = b_0 + b_1·x_1 + b_2·x_2 + ... + b_p·x_p$

Onde:

ŷ → valor previsto

$b_0$ → intercepto

$b_1, b_2, ..., b_p$ → coeficientes das variáveis



In [3]:
X = df[['metros_quadrados', 'quartos', 'banheiros']].values
y = df['preco'].values.reshape(-1,1)

X_treino, X_teste, y_treino, y_teste = train_test_split(
    X, y, test_size=0.3, random_state=42
)

Neste caso temos metros quadrados, quartos e banheiros com X e preço como y. Dividimos o dataaframe com base nisso, e foi feito a divisão entre teste e  treino no metodo holdout.

In [4]:
# adicionando coluna de 1s (intercepto)
X_b = np.c_[np.ones((X_treino.shape[0], 1)), X_treino]

In [5]:
# equação normal
beta = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y_treino

In [6]:
# previsões
X_teste_b = np.c_[np.ones((X_teste.shape[0], 1)), X_teste]
y_pred_manual = X_teste_b @ beta

In [7]:
mse_manual = mean_squared_error(y_teste, y_pred_manual)
rmse_manual = np.sqrt(mse_manual)
r2_manual = r2_score(y_teste, y_pred_manual)
print(f'Modelo Manual - MSE: {mse_manual}, RMSE: {rmse_manual}, R²: {r2_manual}')

Modelo Manual - MSE: 14472.632075198004, RMSE: 120.30225299302587, R²: 0.9999812503570736


# Regressão linear multipla com Scikit-learn

In [8]:
modelo = LinearRegression()
modelo.fit(X_treino, y_treino)

LinearRegression()

In [9]:
y_pred_sklearn = modelo.predict(X_teste)

In [10]:
mse_sklearn = mean_squared_error(y_teste, y_pred_sklearn)
rmse_sklearn = np.sqrt(mse_sklearn)
r2_sklearn = r2_score(y_teste, y_pred_sklearn)

print(f'Modelo Scikit-Learn - MSE: {mse_sklearn}, RMSE: {rmse_sklearn}, R²: {r2_sklearn}')

Modelo Scikit-Learn - MSE: 14472.632075199255, RMSE: 120.30225299303108, R²: 0.9999812503570736


In [11]:
df_metricas = pd.DataFrame({
    'Modelo': ['Manual', 'Scikit-Learn'],
    'MSE': [mse_manual, mse_sklearn],
    'RMSE': [rmse_manual, rmse_sklearn],
    'R2': [r2_manual, r2_sklearn]
})

In [14]:
print("\nCoeficientes Manuais:", beta.flatten())
print("Coeficientes Scikit:", np.r_[modelo.intercept_, modelo.coef_.flatten()])
print("\nMétricas:")
df_metricas


Coeficientes Manuais: [ 291.51282513 1000.00640179  515.79143472  476.50155561]
Coeficientes Scikit: [ 291.51282513 1000.00640179  515.79143472  476.50155561]

Métricas:


,Modelo,MSE,RMSE,R2
0,Manual,14472.632075,120.302253,0.999981
1,Scikit-Learn,14472.632075,120.302253,0.999981


O R² = 0,999981 indica que aproximadamente 99,998% da variação do preço dos imóveis é explicada pelo modelo. Esse valor extremamente próximo de 1 mostra que o ajuste é quase perfeito, ou seja, as variáveis metros quadrados, número de quartos e banheiros explicam muito bem o preço gerado no conjunto de dados.

O RMSE ≈ 120,30 significa que, em média, o erro da previsão é de cerca de R$ 120. Considerando que os preços dos imóveis estão na ordem de dezenas de milhares de reais, esse erro é muito pequeno, tornando o modelo altamente preciso.

# Predição

In [ ]:
novo_imovel = np.array([[1, 80, 3, 2]])  # [1, metros², quartos, banheiros]
preco_manual = novo_imovel @ beta

preco_sklearn = modelo.predict([[80,3,2]])

print("\nPreço previsto (Manual):", preco_manual[0][0])
print("Preço previsto (Scikit):", preco_sklearn[0][0])




Preço previsto (Manual): 82792.40238386496
Preço previsto (Scikit): 82792.40238386502


Os resultados mostram que o modelo de regressão linear múltipla implementado manualmente e o modelo treinado com o Scikit-Learn são praticamente idênticos, tanto nos coeficientes quanto nas métricas de desempenho. Isso confirma que a implementação manual, baseada na forma matricial da equação normal, foi feita corretamente.